# Attendance System Model

In [1]:
#Import Libaries
import cv2
import face_recognition
import numpy as np
from datetime import datetime
import csv
import pickle


In [2]:
def save_model(known_face_encodings, known_face_names, filename='model.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump((known_face_encodings, known_face_names), f)
    print(f"Model saved to {filename}")

def load_model(filename='model.pkl'):
    with open(filename, 'rb') as f:
        known_face_encodings, known_face_names = pickle.load(f)
    print(f"Model loaded from {filename}")
    return known_face_encodings, known_face_names

In [3]:

#  Load and convert Dube image to RGB, then generate the face encoding.
Dube_img = cv2.imread('Student_images/Dube.jpeg')
rgb_img = cv2.cvtColor(Dube_img,cv2.COLOR_BGR2RGB)
Dube_encoding = face_recognition.face_encodings(rgb_img)[0]

In [4]:
Rahul_img = face_recognition.load_image_file('Student_images/Rahul.jpeg')
rgb_img = cv2.cvtColor(Rahul_img,cv2.COLOR_BGR2RGB)

Rahul_encoding = face_recognition.face_encodings(rgb_img)[0]


In [5]:
Priyanka_img = face_recognition.load_image_file('Student_images/Priyanka.jpeg')
rgb_img = cv2.cvtColor(Priyanka_img,cv2.COLOR_BGR2RGB)

Priyanka_encoding = face_recognition.face_encodings(rgb_img)[0]


In [6]:
 # Initialize lists to store face encodings and corresponding names of known individuals.
known_face_encodings=[]
known_face_names=[]

In [7]:
# Append face encodings and corresponding names for Dube, Rahul, and Priyanka to the known face lists.
known_face_encodings.append(Dube_encoding)
known_face_encodings.append(Rahul_encoding)
known_face_encodings.append(Priyanka_encoding)

known_face_names.append("Dube")
known_face_names.append("Rahul")
known_face_names.append("Priyanka")

In [8]:
# Save the model after setting known_face_encodings and known_face_names
save_model(known_face_encodings, known_face_names)

# Load the model before starting the attendance system
known_face_encodings, known_face_names = load_model()

Model saved to model.pkl
Model loaded from model.pkl


In [9]:
known_face_encodings

[array([-0.09077682,  0.09815258, -0.00861273, -0.12080076, -0.14985181,
        -0.05582901, -0.07290477, -0.07338957,  0.1369876 , -0.06659783,
         0.22513019, -0.08815089, -0.20541653, -0.10658409, -0.09205672,
         0.14257059, -0.21329044, -0.13794345,  0.05329   ,  0.04933114,
         0.13474847, -0.02731856,  0.0059605 ,  0.14881299, -0.08836862,
        -0.41588777, -0.06965014, -0.12819168, -0.05888664, -0.0768429 ,
        -0.01351132,  0.01343086, -0.19104604, -0.06072477, -0.02229544,
         0.03851949,  0.02359387, -0.05226441,  0.18553829,  0.02707182,
        -0.27247214,  0.06775616,  0.03086718,  0.26349178,  0.20128901,
         0.04790178,  0.01647726, -0.10128615,  0.13512507, -0.1686542 ,
        -0.00327746,  0.17285651,  0.1167683 , -0.05950103, -0.02766895,
        -0.1018253 ,  0.03835895,  0.12313518, -0.28404665, -0.00560522,
         0.03992122, -0.04617362,  0.03701376, -0.09011234,  0.21618359,
         0.0878024 , -0.10559368, -0.19547047,  0.1

In [10]:
known_face_names

['Dube', 'Rahul', 'Priyanka']

In [11]:
students=known_face_names.copy()

In [12]:
face_names=[]

In [13]:
# Generate a filename based on the current date, open the CSV file for writing, and prepare a CSV writer.
current_date = datetime.now().strftime('%Y-%m-%d')
filename = f'attendance_{current_date}.csv'
f= open(filename, 'w+', newline='')
lnwriter = csv.writer(f)

In [14]:
video_cap= cv2.VideoCapture(0)

In [15]:
while True:
    ret, frame= video_cap.read()
    small_frame = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])

    face_locations =face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame,face_locations)

    for(top,right,bottom,left), face_encodings in zip(face_locations,face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings,face_encodings)
        name = 'unknown'

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]
        
        face_names.append(name)
        if name in known_face_names:
            if name in students:
                students.remove(name)
                current_time = datetime.now().strftime("%H-%M-%S")
                lnwriter.writerow([name, current_time,'Present'])
        
        cv2.rectangle(frame,(left,top),(right,bottom),(0,0,255),2)
        cv2.putText(frame,name,(left,top-10),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,0,255),2)
        
    

    # Display the resulting frame
    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_cap.release()
cv2.destroyAllWindows()
f.close()
